In [3]:
import tensorflow.keras as keras
import tensorflow as tf

from tensorflow.keras.models import load_model
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.config.experimental_list_devices())


['/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:XLA_GPU:0', '/job:localhost/replica:0/task:0/device:XLA_CPU:0', '/job:localhost/replica:0/task:0/device:GPU:0']


In [4]:
tf.__version__

'2.0.0-dev20190506'

In [5]:
tf.executing_eagerly()

True

In [6]:
batch_size = 32

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [7]:
from tensorflow.keras.models import load_model

In [8]:
model = tf.keras.applications.VGG16(weights='imagenet')

In [7]:
model.save('./baseline_vgg_imagenet.h5')

In [9]:
targets = [i for i, layer in enumerate(model.layers) if layer.__class__.__name__ == 'Conv2D']

In [8]:
targets

[1, 2, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17]

In [10]:

get_output = tf.keras.Model(inputs=model.input, 
                            outputs=[model.layers[target - 1].output,
                                     model.layers[target].output])

NameError: name 'target' is not defined

In [12]:
get_output.output[1].shape[-1]

NameError: name 'get_output' is not defined

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

dataset_test = dataset.batch(batch_size)
dataset_test = dataset.repeat()

NameError: name 'x_train' is not defined

In [11]:
def build_replacement(get_output):
    inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=get_output.output[1].shape[-1], 
                                        kernel_size= (3,3),
                                        padding='Same')(inputs)
    X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    
    build_replacement.counter += 1
    
    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=get_output.output[1].shape[-1],
                                        kernel_size=(3,3), 
                                        padding='Same')(X)
    X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)
    
    build_replacement.counter += 1
    
    return replacement_layers

build_replacement.counter = 0

In [9]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(50000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(10000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [10]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from numba import cuda


In [12]:


import gc
while len(targets) > 1:
    
    print(f'targets {targets}')
    print("taking target")
    target = targets[1]
    
    print(f'making output for target layer {target}')
    
    get_output = tf.keras.Model(inputs=model.input, 
                                outputs=[model.layers[target - 1].output,
                                         model.layers[target].output])
    
    print(f'making replacement layers for target layer {target}')
    replacement_layers = build_replacement(get_output)
    
    replacement_len = len(replacement_layers.layers)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=.001)

    loss_object = tf.losses.MeanSquaredError()
    
    replacement_layers.compile(loss=loss_object, optimizer=optimizer)
    
    save = tf.keras.callbacks.ModelCheckpoint('./replacement_layer.h5', 
                                              verbose=0, 
                                              save_weights_only=True,
                                              save_best_only=True)

    
    print(f'starting fit generator for target layer {target}')
#     replacement_layers.fit_generator(generator=train_gen, 
#                                      epochs=50, 
#                                      validation_data=test_gen ,
#                                      verbose=0, callbacks=[save])
    
    print('saving replacement layers to json')
    
#     replacement_json = replacement_layers.to_json()
#     with open('replacement_layer.json', 'w') as json_file:
#         json_file.write(replacement_json)
        
#     del replacement_layers
    
#     with open('replacement_layer.json', 'r') as json_file:
#         replacement_layers = tf.keras.models.model_from_json(json_file.read())

#     print('loading replacement layers weights')
#     replacement_layers.load_weights('replacement_layer.h5')
#     replacement_layers.compile(loss=loss_object, optimizer=optimizer)
#     replacement_layers.evaluate_generator(test_gen)
    # build top half of model
    print('building top half of model')
    get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[target - 1].output])
    # add in replacement layers
    print('building middle of model with replacement layers')
    new_joint = tf.keras.Model(inputs=get_output.input, outputs=replacement_layers(get_output.output))
    
    #new_joint.summary()
    
    # build bottom of model
    bottom_half = tf.keras.Sequential()
    for layer in model.layers[target + 1::]:
        bottom_half.add(layer)
        
    
    
    print('building bottom of model')
    bottom_half.build(input_shape=new_joint.output.shape)
    bottom_half.summary()
    print('combining model')
    combined = tf.keras.Model(inputs=new_joint.input, outputs=bottom_half(new_joint.output))
    
    combined.layers[-1].trainable=False
    opt = keras.optimizers.RMSprop(lr=0.00005, decay=1e-6)
    combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    
    #combined.summary()
    del bottom_half, new_joint, replacement_layers, model
    gc.collect()
    
#     print('testing combined model')
#     scores = combined.evaluate(x_test, y_test, verbose=1)
#     print('Test loss:', scores[0])
#     print('Test accuracy:', scores[1])
    
    new_combined = tf.keras.Sequential()
    new_layers = []
    new_combined.add(tf.keras.layers.Input(shape=(224,224,3)))
    accum = 0
    print('refactoring model')
    for layer in combined.layers:
        #print(layer.__class__.__name__)
        if hasattr(layer, 'layers'):
            
            for sublayer in layer.layers:
                #print(sublayer.__class__)
                if(sublayer.__class__.__name__ != 'InputLayer'): 
                    new_layers.append((sublayer.__class__.__name__, sublayer.get_config(), accum))

                accum += 1
        elif layer.__class__.__name__ != 'InputLayer':          
            new_layers.append((layer.__class__.__name__, layer.get_config(), accum))
                
            accum += 1 




    for i, layer in enumerate(new_layers):
        new_combined.add(keras.layers.deserialize(
                                {'class_name': layer[0], 
                                 'config': layer[1]}))

    new_combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

   

    accum = 0
    for i, layer in enumerate(combined.layers):
        if hasattr(layer, 'layers'):

            for sublayer in layer.layers:
                #print(f'{accum} sub is {sublayer} new is {new_combined.layers[accum]}')
                if(sublayer.__class__.__name__ != 'InputLayer'): 
                    #print(sublayer.__class__)
                    new_combined.layers[accum].set_weights(sublayer.get_weights())
                    accum += 1
    #             else:
    #                 accum += 1
            continue
        else:
            #print(layer)
            if(layer.__class__.__name__ != 'InputLayer'):
                print()
                new_combined.layers[accum].set_weights(layer.get_weights())
                accum +=1 
            elif(layer.__class__.__name__ == 'Flatten'):
                accum += 1

    print('freezing first half of layers')
    for i in range(target):
        new_combined.layers[i].trainable = False
    
    print('freezing last half of layers')
    for i in range(target +  replacement_len, len(new_combined.layers)):
        new_combined.layers[i].trainable = False
        
    new_combined.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    del combined
    gc.collect()

    new_save=tf.keras.callbacks.ModelCheckpoint('./refactor_finetune.h5', 
                                                verbose=1, 
                                                save_weights_only=False, 
                                                save_best_only=True)
    print('fine tuning combined model')
    #new_combined.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=5, callbacks=[new_save])
#     new_combined.fit_generator(datagen.flow(x_train, y_train,
#                                      batch_size=batch_size),
#                         epochs=5,
#                         validation_data=(x_test, y_test),
#                         #workers=5,
#                         callbacks=[new_save])
    
    print('loading best weights from fine tune')
    #new_combined.load_weights('./refactor_finetune.h5')
    #new_combined.save('.refactor_finetune.h5')

    model = tf.keras.Model(inputs=new_combined.input, 
                                outputs=new_combined.output)
    model.summary()
    del new_combined
    gc.collect()
    model.save('./refactor_finetune.h5')
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model('./refactor_finetune.h5')
    print('new summary')
    model.summary()
    targets = [i for i, layer in enumerate(model.layers) if layer.__class__.__name__ == 'Conv2D']
    
    

targets [1, 2, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17]
taking target
making output for target layer 2
making replacement layers for target layer 2
starting fit generator for target layer 2
saving replacement layers to json
building top half of model
building middle of model with replacement layers
building bottom of model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_______________________________________________________________

W0506 19:57:13.408628 140377590093632 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


new summary
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
sep_conv_0 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 224, 224, 64)      256       
_________________________________________________________________
relu_0 (ReLU)                (None, 224, 224, 64)      0         
_________________________________________________________________
sep_conv_1 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 224, 224, 6

W0506 19:57:19.759829 140377590093632 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


new summary
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
sep_conv_0 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 224, 224, 64)      256       
_________________________________________________________________
relu_0 (ReLU)                (None, 224, 224, 64)      0         
_________________________________________________________________
sep_conv_1 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 224, 224, 6

...

new summary
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
sep_conv_0 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 224, 224, 64)      256       
_________________________________________________________________
relu_0 (ReLU)                (None, 224, 224, 64)      0         
_________________________________________________________________
sep_conv_1 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 224, 224, 6

W0506 19:58:26.406017 140377590093632 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


new summary
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
sep_conv_0 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 224, 224, 64)      256       
_________________________________________________________________
relu_0 (ReLU)                (None, 224, 224, 64)      0         
_________________________________________________________________
sep_conv_1 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 224, 224, 6

W0506 19:58:35.000385 140377590093632 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


new summary
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
sep_conv_0 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 224, 224, 64)      256       
_________________________________________________________________
relu_0 (ReLU)                (None, 224, 224, 64)      0         
_________________________________________________________________
sep_conv_1 (SeparableConv2D) (None, 224, 224, 64)      4736      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 224, 224, 6